In [1]:
from pymongo import MongoClient
from datetime import datetime
import numpy as np
import pandas as pd
import csv

In [2]:
global arrPortNames, arrKY_CD, arrOFNS_DESC, arrPD_CD, arrPD_DESC
global arrPedesYear, arrPedesMonth
global arrCrimeDbBoroughNames, arrPedesDbBoroughNames, arrPREM_DESC_TYPE
global FIELDNAMES, outputCSVfile

In [3]:
arrPedesYear = [2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
arrPedesMonth = [5, 9]

arrCrimeDbBoroughNames = [ "QUEENS", "BRONX", "BROOKLYN", "STATEN ISLAND", "MANHATTAN" ]
arrPedesDbBoroughNames = [ "Queens", "Bronx", "Brooklyn", "Staten Island", "Manhattan" ]

arrPortsDbPortNames = [ "new york", "newark", "los angeles" ]

In [4]:
client = MongoClient('localhost:27017')
database = client['db1story']
coll1CrimeData = database['coll1nyccrimedata']
coll2PedesData = database['coll2pedesdata']
coll3PortData = database['coll3portsdata']

In [6]:
## processing for pedes info + port info + total crimes + KY_CDs + OFNS_DESC + PD_CD + PD_DESC in the month   for testing
#  pipeline 1 to 7

In [ ]:
# process the PD_DESC data and store it -- pipeline 7
# setup and process for month, year, boro and PD_DESC and store it
# db.coll1nyccrimedata.aggregate([ {"$match": {"PD_DESC": "ASSAULT 2,1,UNCLASSIFIED", "BORO_NM": "QUEENS", "CMPLNT_FR_DT": {"$regex": "^05[/][0-9]{2}[/]2007$" } } } , {"$group": {"_id": 1, "TotalCrimesForPD_CDForMonthYearBoro": {"$sum": 1}}} ]) --- get the document for May'2007, specific boro, for the specific PD_DESC   -- get answer = { "_id" : 1, "TotalCrimesForPD_CDForMonthYearBoro" : 282 }
# db.coll1nyccrimedata.distinct("PD_DESC", {"BORO_NM": "QUEENS", "CMPLNT_FR_DT": {$regex: '^05[/][0-9]{2}[/]2007$' } })  -- answer = [ "ASSAULT 2,1,UNCLASSIFIED", "HARASSMENT,SUBD 1,CIVILIAN", "BURGLARY,RESIDENCE,NIGHT", and many more items ]
pipeline7 = [ {"$match": {"PD_DESC": "garbage", "BORO_NM": "garbage", "CMPLNT_FR_DT": {"$regex": "garbage" } } } , {"$group": {"_id": 1, "TotalCrimesForPD_DESCForMonthYearBoro": {"$sum": 1}}} ]

In [10]:
arrKY_CD = [106, 101] #, 109 ]
arrOFNS_DESC = ['FELONY ASSAULT' ] #, 'HARRASSMENT 2', 'BURGLARY' ]
arrPD_CD = [ 109, 637 ] # 638, 223 ]
arrPD_DESC = [ "ASSAULT 2,1,UNCLASSIFIED", "HARASSMENT,SUBD 1,CIVILIAN", "BURGLARY,RESIDENCE,NIGHT" ]

In [13]:
ARRrowDataToWrite = []
for currPedesBoroBeingProcessed in arrPedesDbBoroughNames[:1]:
    for currPedesYearBeingProcessed in arrPedesYear[:1]:
        for currPedesMonthBeingProcessed in arrPedesMonth[:1]:
            print(f"\n\n*** **** **** ****\nProcessing for:\nYear={currPedesYearBeingProcessed}\tMonth={currPedesMonthBeingProcessed}\t\tBoro={currPedesBoroBeingProcessed}")
            #logging.warning(f"\n\n*** **** **** ****\nProcessing for:\nYear={currPedesYearBeingProcessed}\tMonth={currPedesMonthBeingProcessed}\t\tBoro={currPedesBoroBeingProcessed}")
            rowDataToWrite = []
            rowDataToWrite.append(currPedesYearBeingProcessed)
            rowDataToWrite.append(currPedesMonthBeingProcessed)
            rowDataToWrite.append(currPedesBoroBeingProcessed)
            
            # read the count of pedestrians and store it
            # -- db.coll2pedesdata.aggregate([ {"$match": {"year": 2007, "month": 5, "boroughBridgeName": "Queens"} } , {"$group": {"_id": 1, "TotalPedesCount": {"$sum": "$count"}}} ]) --- shows 199427 count of pedes summed
            pipeline1 = [ {"$match": {"year": 0, "month": 0, "boroughBridgeName": "garbage"} } , {"$group": {"_id": 1, "TotalPedesCount": {"$sum": "$count"}}} ]

            pipeline1[0]['$match']['year'] = currPedesYearBeingProcessed
            pipeline1[0]['$match']['month'] = currPedesMonthBeingProcessed
            pipeline1[0]['$match']['boroughBridgeName'] = currPedesBoroBeingProcessed
            cursorTotalPedesCount = coll2PedesData.aggregate(pipeline1)
            for cursor in cursorTotalPedesCount:
                rowDataToWrite.append(cursor["TotalPedesCount"])
            cursorTotalPedesCount.close()
            print(f"Cur 1 done")
            
            # process the ports data now
            # setup and read the port passenger entry count and store it
            # -- db.coll3portsdata.aggregate([ {"$match": {"portName": "new york", "year" : 2007, "month" : 5} } ]) --- get the document for May'2007
            for currPortBeingProcessed in arrPortsDbPortNames:
                pipeline2 = [ {"$match": {"portName": "garbage", "year" : 0, "month" : 0} } ] 
                pipeline2[0]['$match']['year'] = currPedesYearBeingProcessed
                pipeline2[0]['$match']['month'] = currPedesMonthBeingProcessed
                pipeline2[0]['$match']['portName'] = currPortBeingProcessed
                rowDataToWrite.append(currPortBeingProcessed)
                cursorTotalPortEntryCount = coll3PortData.aggregate(pipeline2)
                for cursor in cursorTotalPortEntryCount:
                    rowDataToWrite.append(cursor["count"])
                cursorTotalPortEntryCount.close()
            print(f"Cur 2 done")
            
            # process for the total count of crimes in the month, year, boro and store it
            # setup process total crimes count for the month, year, boro
            # -- db.coll1nyccrimedata.find({"BORO_NM": "QUEENS", "CMPLNT_FR_DT": {$regex: /^05[/][0-9]{2}[/]2007$/}}).count()   -- gave answer 9214
            # -- db.coll1nyccrimedata.aggregate([ {"$match": {"BORO_NM": "QUEENS", "CMPLNT_FR_DT": {$regex: /^05[/][0-9]{2}[/]2007$/} } } , {"$group": {"_id": 1, "TotalCrimesForMonthYearBoro": {"$sum": 1}}} ])  -- gave answer { "_id" : 1, "TotalCrimesForMonthYearBoro" : 9214 }
            pipeline3 = [ {"$match": {"BORO_NM": "garbage", "CMPLNT_FR_DT": {"$regex": "garbage"} } } , {"$group": {"_id": 1, "TotalCrimesForMonthYearBoro": {"$sum": 1}}} ]
            #regexString = '/^' + mymonth + '[/][\d]{2}[/]' + myyear + '$/'
            #newRegexString = '/^' + str(currPedesMonthBeingProcessed).zfill(2) + '[/][0-9]{2}[/]' + str(currPedesYearBeingProcessed) + '$/'
            newRegexString = '^' + str(currPedesMonthBeingProcessed).zfill(2) + '[/][0-9]{2}[/]' + str(currPedesYearBeingProcessed) + '$'
            #print(f"\nnewRegexString=\n{newRegexString}")
            #print(f"before pipeline3=\n{pipeline3}")
            pipeline3[0]['$match']['CMPLNT_FR_DT']['$regex'] = newRegexString
            pipeline3[0]['$match']['BORO_NM'] = currPedesBoroBeingProcessed.upper()
            #print(f"query pipeline3=\n{pipeline3}")
            cursorTotalCrimesInMonthYearBoroCount = coll1CrimeData.aggregate(pipeline3)
            for cursor in cursorTotalCrimesInMonthYearBoroCount:
                rowDataToWrite.append(cursor["TotalCrimesForMonthYearBoro"])
            cursorTotalCrimesInMonthYearBoroCount.close()
            print(f"Cur 3 done")
            
            # setup and process for month, year, boro and KY_CD and store it
            # db.coll1nyccrimedata.aggregate([ {"$match": {"KY_CD": 106, "BORO_NM": "QUEENS", "CMPLNT_FR_DT": {"$regex": "^05[/][0-9]{2}[/]2007$" } } } , {"$group": {"_id": 1, "TotalCrimesForKY_CDForMonthYearBoro": {"$sum": 1}}} ]) --- get the document for May'2007, specific boro, for the specific KY_CD   -- get answer = { "_id" : 1, "TotalCrimesForKY_CDForMonthYearBoro" : 324 }
            # db.coll1nyccrimedata.aggregate( [ {"$match": {"KY_CD": 106, "BORO_NM": "QUEENS", "CMPLNT_FR_DT": {"$regex": "^05[/][0-9]{2}[/]2007$" } } } , {"$group": {"_id": 1, "TotalCrimesForKY_CDForMonthYearBoro": {"$sum": 1}}} ] )
            # db.coll1nyccrimedata.distinct("BORO_NM", {"KY_CD": 106, "CMPLNT_FR_DT": {$regex: '^05[/][0-9]{2}[/]2007$' } })  -- [ "QUEENS", "BROOKLYN", "MANHATTAN", "BRONX", null, "STATEN ISLAND" ]
            for currKY_CDBeingProcessed in arrKY_CD:
                print(f"\nProcessing for:\nKY_CD={currKY_CDBeingProcessed}")
                #logging.warning(f"\nProcessing for:\nKY_CD={currKY_CDBeingProcessed}")
                pipeline4 = [ {"$match": {"KY_CD": 0, "BORO_NM": "garbage", "CMPLNT_FR_DT": {"$regex": "garbage" } } } , {"$group": {"_id": 1, "TotalCrimesForKY_CDForMonthYearBoro": {"$sum": 1}}} ]
                newRegexString = '^' + str(currPedesMonthBeingProcessed).zfill(2) + '[/][0-9]{2}[/]' + str(currPedesYearBeingProcessed) + '$'
                pipeline4[0]['$match']['CMPLNT_FR_DT']['$regex'] = newRegexString
                pipeline4[0]['$match']['BORO_NM'] = currPedesBoroBeingProcessed.upper()
                pipeline4[0]['$match']['KY_CD'] = currKY_CDBeingProcessed
                print(f"query pipeline4=\n{pipeline4}")
                rowDataToWrite.append(currKY_CDBeingProcessed)
                cursorTotalCrimesInKY_CDMonthYearBoroCount = coll1CrimeData.aggregate(pipeline4)
                toAppendValue = 0
                for cursor in cursorTotalCrimesInKY_CDMonthYearBoroCount:
                    print(f"     entered for cursor\t cursor={cursor}")
                    try:
                        toAppendValue = cursor["TotalCrimesForKY_CDForMonthYearBoro"]
                    except KeyError:
                        print(f"\n\t\tKeyError for KY_CD={currKY_CDBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        print(f"\t\tSo inserted 0 for the crime count and continued.")
                        #logging.warning(f"\n\t\tKeyError for KY_CD={currKY_CDBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        #logging.warning(f"\n\t\tSo inserted 0 for the crime count and continued.")
                        toAppendValue = 0
                    except Exception as e:
                        cursorTotalCrimesInKY_CDMonthYearBoroCount.close()
                        print(f"\nERROR: Unhandled exception processing for:\n\t\tKeyError for KY_CD={currKY_CDBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        print(f"ERROR: Error details = {e}")
                        #logging.error(f"\nERROR: Unhandled exception processing for:\n\t\tKeyError for KY_CD={currKY_CDBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        #logging.error(f"ERROR: Error details = {e}")
                    cursorTotalCrimesInKY_CDMonthYearBoroCount.close()
                rowDataToWrite.append(toAppendValue)
            print(f"Cur 4 done")
            
            # process the OFNS_DESC data and store it -- pipeline 5
            # setup and process for month, year, boro and OFNS_DESC and store it
            # db.coll1nyccrimedata.aggregate([ {"$match": {"OFNS_DESC": "FELONY ASSAULT", "BORO_NM": "QUEENS", "CMPLNT_FR_DT": {"$regex": "^05[/][0-9]{2}[/]2007$" } } } , {"$group": {"_id": 1, "TotalCrimesForOFNS_DESCForMonthYearBoro": {"$sum": 1}}} ]) --- get the document for May'2007, specific boro, for the specific OFNS_DESC   -- get answer = { "_id" : 1, "TotalCrimesForOFNS_DESCForMonthYearBoro" : 324 }
            # db.coll1nyccrimedata.distinct("OFNS_DESC", {"BORO_NM": "QUEENS", "CMPLNT_FR_DT": {$regex: '^05[/][0-9]{2}[/]2007$' } })  -- answer = arrOFNS_DESC = ['FELONY ASSAULT', 'HARRASSMENT 2', 'BURGLARY', and more items totalling to 40 approx ]
            for currOFNS_DESCBeingProcessed in arrOFNS_DESC:
                print(f"\nProcessing for:\nOFNS_DESC={currOFNS_DESCBeingProcessed}")
                #logging.warning(f"\nProcessing for:\nOFNS_DESC={currOFNS_DESCBeingProcessed}")
                pipeline5 = [ {"$match": {"OFNS_DESC": "garbage", "BORO_NM": "garbage", "CMPLNT_FR_DT": {"$regex": "garbage" } } } , {"$group": {"_id": 1, "TotalCrimesForOFNS_DESCForMonthYearBoro": {"$sum": 1}}} ]
                newRegexString = '^' + str(currPedesMonthBeingProcessed).zfill(2) + '[/][0-9]{2}[/]' + str(currPedesYearBeingProcessed) + '$'
                pipeline5[0]['$match']['CMPLNT_FR_DT']['$regex'] = newRegexString
                pipeline5[0]['$match']['BORO_NM'] = currPedesBoroBeingProcessed.upper()
                pipeline5[0]['$match']['OFNS_DESC'] = currOFNS_DESCBeingProcessed
                print(f"query pipeline5=\n{pipeline5}")
                rowDataToWrite.append(currOFNS_DESCBeingProcessed)
                cursorTotalCrimesInOFNS_DESCMonthYearBoroCount = coll1CrimeData.aggregate(pipeline5)
                toAppendValue = 0
                for cursor in cursorTotalCrimesInOFNS_DESCMonthYearBoroCount:
                    print(f"     entered for cursor\t cursor={cursor}")
                    try:
                        toAppendValue = cursor["TotalCrimesForOFNS_DESCForMonthYearBoro"]
                    except KeyError:
                        print(f"\n\t\tKeyError for OFNS_DESC={currOFNS_DESCBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        print(f"\t\tSo inserted 0 for the crime count and continued.")
                        #logging.warning(f"\n\t\tKeyError for OFNS_DESC={currOFNS_DESCBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        #logging.warning(f"\n\t\tSo inserted 0 for the crime count and continued.")
                        toAppendValue = 0
                    except Exception as e:
                        cursorTotalCrimesInOFNS_DESCMonthYearBoroCount.close()
                        print(f"\nERROR: Unhandled exception processing for:\n\t\tKeyError for OFNS_DESC={currOFNS_DESCBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        print(f"ERROR: Error details = {e}")
                        logging.error(f"\nERROR: Unhandled exception processing for:\n\t\tKeyError for OFNS_DESC={currOFNS_DESCBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        logging.error(f"ERROR: Error details = {e}")
                cursorTotalCrimesInOFNS_DESCMonthYearBoroCount.close()
                rowDataToWrite.append(toAppendValue)
            print(f"Cur 5 done")
            
            # process the PD_CD data and store it -- pipeline 6
            # setup and process for month, year, boro and PD_CD and store it
            # db.coll1nyccrimedata.aggregate([ {"$match": {"PD_CD": 109, "BORO_NM": "QUEENS", "CMPLNT_FR_DT": {"$regex": "^05[/][0-9]{2}[/]2007$" } } } , {"$group": {"_id": 1, "TotalCrimesForPD_CDForMonthYearBoro": {"$sum": 1}}} ]) --- get the document for May'2007, specific boro, for the specific PD_CD   -- get answer = { "_id" : 1, "TotalCrimesForPD_CDForMonthYearBoro" : 282 }
            # db.coll1nyccrimedata.distinct("PD_CD", {"BORO_NM": "QUEENS", "CMPLNT_FR_DT": {$regex: '^05[/][0-9]{2}[/]2007$' } })  -- answer = [ 109, 637, 638, 223, and many more items ]
            for currPD_CDBeingProcessed in arrPD_CD:
                print(f"\nProcessing for:\nPD_CD={currPD_CDBeingProcessed}")
                #logging.warning(f"\nProcessing for:\nPD_CD={currPD_CDBeingProcessed}")
                pipeline6 = [ {"$match": {"PD_CD": 0, "BORO_NM": "garbage", "CMPLNT_FR_DT": {"$regex": "garbage" } } } , {"$group": {"_id": 1, "TotalCrimesForPD_CDForMonthYearBoro": {"$sum": 1}}} ]
                newRegexString = '^' + str(currPedesMonthBeingProcessed).zfill(2) + '[/][0-9]{2}[/]' + str(currPedesYearBeingProcessed) + '$'
                pipeline6[0]['$match']['CMPLNT_FR_DT']['$regex'] = newRegexString
                pipeline6[0]['$match']['BORO_NM'] = currPedesBoroBeingProcessed.upper()
                pipeline6[0]['$match']['PD_CD'] = currPD_CDBeingProcessed
                print(f"query pipeline6=\n{pipeline6}")
                rowDataToWrite.append(currPD_CDBeingProcessed)
                cursorTotalCrimesInPD_CDMonthYearBoroCount = coll1CrimeData.aggregate(pipeline6)
                toAppendValue = 0
                for cursor in cursorTotalCrimesInPD_CDMonthYearBoroCount:
                    print(f"     entered for cursor\t cursor={cursor}")
                    try:
                        toAppendValue = cursor["TotalCrimesForPD_CDForMonthYearBoro"]
                    except KeyError:
                        print(f"\n\t\tKeyError for PD_CD={currPD_CDBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        print(f"\t\tSo inserted 0 for the crime count and continued.")
                        #logging.warning(f"\n\t\tKeyError for PD_CD={currPD_CDBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        #logging.warning(f"\n\t\tSo inserted 0 for the crime count and continued.")
                        toAppendValue = 0
                    except Exception as e:
                        cursorTotalCrimesInPD_CDMonthYearBoroCount.close()
                        print(f"\nERROR: Unhandled exception processing for:\n\t\tKeyError for PD_CD={currPD_CDBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        print(f"ERROR: Error details = {e}")
                        #logging.error(f"\nERROR: Unhandled exception processing for:\n\t\tKeyError for PD_CD={currPD_CDBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        #logging.error(f"ERROR: Error details = {e}")
                cursorTotalCrimesInPD_CDMonthYearBoroCount.close()
                rowDataToWrite.append(toAppendValue)
            print(f"Cur 6 done")
            
            # process the PD_DESC data and store it -- pipeline 7
            # setup and process for month, year, boro and PD_DESC and store it
            # db.coll1nyccrimedata.aggregate([ {"$match": {"PD_DESC": "ASSAULT 2,1,UNCLASSIFIED", "BORO_NM": "QUEENS", "CMPLNT_FR_DT": {"$regex": "^05[/][0-9]{2}[/]2007$" } } } , {"$group": {"_id": 1, "TotalCrimesForPD_DESCForMonthYearBoro": {"$sum": 1}}} ]) --- get the document for May'2007, specific boro, for the specific PD_DESC   -- get answer = { "_id" : 1, "TotalCrimesForPD_DESCForMonthYearBoro" : 282 }
            # db.coll1nyccrimedata.distinct("PD_DESC", {"BORO_NM": "QUEENS", "CMPLNT_FR_DT": {$regex: '^05[/][0-9]{2}[/]2007$' } })  -- answer = [ "ASSAULT 2,1,UNCLASSIFIED", "HARASSMENT,SUBD 1,CIVILIAN", "BURGLARY,RESIDENCE,NIGHT", and many more items ]
            for currPD_DESCBeingProcessed in arrPD_DESC:
                print(f"\nProcessing for:\nPD_DESC={currPD_DESCBeingProcessed}")
                #logging.warning(f"\nProcessing for:\nPD_DESC={currPD_DESCBeingProcessed}")
                pipeline7 = [ {"$match": {"PD_DESC": "garbage", "BORO_NM": "garbage", "CMPLNT_FR_DT": {"$regex": "garbage" } } } , {"$group": {"_id": 1, "TotalCrimesForPD_DESCForMonthYearBoro": {"$sum": 1}}} ]
                newRegexString = '^' + str(currPedesMonthBeingProcessed).zfill(2) + '[/][0-9]{2}[/]' + str(currPedesYearBeingProcessed) + '$'
                pipeline7[0]['$match']['CMPLNT_FR_DT']['$regex'] = newRegexString
                pipeline7[0]['$match']['BORO_NM'] = currPedesBoroBeingProcessed.upper()
                pipeline7[0]['$match']['PD_DESC'] = currPD_DESCBeingProcessed
                #print(f"query pipeline7=\n{pipeline7}")
                rowDataToWrite.append(currPD_DESCBeingProcessed)
                cursorTotalCrimesInPD_DESCMonthYearBoroCount = coll1CrimeData.aggregate(pipeline7)
                toAppendValue = 0
                for cursor in cursorTotalCrimesInPD_DESCMonthYearBoroCount:
                    print(f"     entered for cursor\t cursor={cursor}")
                    try:
                        toAppendValue = cursor["TotalCrimesForPD_DESCForMonthYearBoro"]
                    except KeyError:
                        print(f"\n\t\tKeyError for PD_DESC={currPD_DESCBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        print(f"\t\tSo inserted 0 for the crime count and continued.")
                        #logging.warning(f"\n\t\tKeyError for PD_DESC={currPD_DESCBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        #logging.warning(f"\n\t\tSo inserted 0 for the crime count and continued.")
                        toAppendValue = 0
                    except Exception as e:
                        cursorTotalCrimesInPD_DESCMonthYearBoroCount.close()
                        print(f"\nERROR: Unhandled exception processing for:\n\t\tKeyError for PD_DESC={currPD_DESCBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        print(f"ERROR: Error details = {e}")
                        #logging.error(f"\nERROR: Unhandled exception processing for:\n\t\tKeyError for PD_DESC={currPD_DESCBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        #logging.error(f"ERROR: Error details = {e}")
                cursorTotalCrimesInPD_DESCMonthYearBoroCount.close()
                rowDataToWrite.append(toAppendValue)
            print(f"Cur 7 done")
            #
            #####################
            ARRrowDataToWrite.append(rowDataToWrite)
   



*** **** **** ****
Processing for:
Year=2007	Month=5		Boro=Queens
Cur 1 done
Cur 2 done
Cur 3 done

Processing for:
KY_CD=106
query pipeline4=
[{'$match': {'KY_CD': 106, 'BORO_NM': 'QUEENS', 'CMPLNT_FR_DT': {'$regex': '^05[/][0-9]{2}[/]2007$'}}}, {'$group': {'_id': 1, 'TotalCrimesForKY_CDForMonthYearBoro': {'$sum': 1}}}]
     entered for cursor	 cursor={'_id': 1, 'TotalCrimesForKY_CDForMonthYearBoro': 324}

Processing for:
KY_CD=101
query pipeline4=
[{'$match': {'KY_CD': 101, 'BORO_NM': 'QUEENS', 'CMPLNT_FR_DT': {'$regex': '^05[/][0-9]{2}[/]2007$'}}}, {'$group': {'_id': 1, 'TotalCrimesForKY_CDForMonthYearBoro': {'$sum': 1}}}]
Cur 4 done

Processing for:
OFNS_DESC=FELONY ASSAULT
query pipeline5=
[{'$match': {'OFNS_DESC': 'FELONY ASSAULT', 'BORO_NM': 'QUEENS', 'CMPLNT_FR_DT': {'$regex': '^05[/][0-9]{2}[/]2007$'}}}, {'$group': {'_id': 1, 'TotalCrimesForOFNS_DESCForMonthYearBoro': {'$sum': 1}}}]
     entered for cursor	 cursor={'_id': 1, 'TotalCrimesForOFNS_DESCForMonthYearBoro': 324}
Cu

In [14]:
print(ARRrowDataToWrite)

[[2007, 5, 'Queens', 199247, 'new york', 306510, 'newark', 118361, 'los angeles', 196387, 9214, 106, 324, 101, 0, 'FELONY ASSAULT', 324, 109, 282, 637, 532, 'ASSAULT 2,1,UNCLASSIFIED', 282, 'HARASSMENT,SUBD 1,CIVILIAN', 532, 'BURGLARY,RESIDENCE,NIGHT', 105]]


In [ ]:
## processing for pedes info + port info + total crimes + KY_CDs + OFNS_DESC + PD_CD + PD_DESC + PREM_TYP_DESC + LAW_CAT_CD in the month   for testing
#  pipeline 1 to 9

In [ ]:
# process the PREM_TYP_DESC data and store it -- pipeline 8
# setup and process for month, year, boro and PREM_TYP_DESC and store it
# db.coll1nyccrimedata.aggregate([ {"$match": {"PREM_TYP_DESC": "STREET", "BORO_NM": "QUEENS", "CMPLNT_FR_DT": {"$regex": "^05[/][0-9]{2}[/]2007$" } } } , {"$group": {"_id": 1, "TotalCrimesForPREM_TYP_DESCForMonthYearBoro": {"$sum": 1}}} ]) --- get the document for May'2007, specific boro, for the specific PREM_TYP_DESC   -- get answer = { "_id" : 1, "TotalCrimesForPREM_TYP_DESCForMonthYearBoro" : 3159 }
# db.coll1nyccrimedata.distinct("PREM_TYP_DESC", {"BORO_NM": "QUEENS", "CMPLNT_FR_DT": {$regex: '^05[/][0-9]{2}[/]2007$' } })  -- answer = ["BAR/NIGHT CLUB", "STREET", "RESIDENCE-HOUSE", and many more items ]
pipeline8 = [ {"$match": {"PREM_TYP_DESC": "garbage", "BORO_NM": "garbage", "CMPLNT_FR_DT": {"$regex": "garbage" } } } , {"$group": {"_id": 1, "TotalCrimesForPREM_TYP_DESCForMonthYearBoro": {"$sum": 1}}} ]

# process the LAW_CAT_CD data and store it -- pipeline 9
# setup and process for month, year, boro and LAW_CAT_CD and store it
# db.coll1nyccrimedata.aggregate([ {"$match": {"LAW_CAT_CD": "FELONY", "BORO_NM": "QUEENS", "CMPLNT_FR_DT": {"$regex": "^05[/][0-9]{2}[/]2007$" } } } , {"$group": {"_id": 1, "TotalCrimesForLAW_CAT_CDForMonthYearBoro": {"$sum": 1}}} ]) --- get the document for May'2007, specific boro, for the specific LAW_CAT_CD   -- get answer = { "_id" : 1, "TotalCrimesForLAW_CAT_CDForMonthYearBoro" : 3006 }
# db.coll1nyccrimedata.distinct("LAW_CAT_CD", {"BORO_NM": "QUEENS", "CMPLNT_FR_DT": {$regex: '^05[/][0-9]{2}[/]2007$' } })  -- answer = [ "FELONY", "VIOLATION", "MISDEMEANOR" ]
pipeline9 = [ {"$match": {"LAW_CAT_CD": "garbage", "BORO_NM": "garbage", "CMPLNT_FR_DT": {"$regex": "garbage" } } } , {"$group": {"_id": 1, "TotalCrimesForLAW_CAT_CDForMonthYearBoro": {"$sum": 1}}} ]

In [5]:
arrKY_CD = [106, 101] #, 109 ]
arrOFNS_DESC = ['FELONY ASSAULT' ] #, 'HARRASSMENT 2', 'BURGLARY' ]
arrPD_CD = [ 109, 637 ] # 638, 223 ]
arrPD_DESC = [ "ASSAULT 2,1,UNCLASSIFIED" ] #, "HARASSMENT,SUBD 1,CIVILIAN", "BURGLARY,RESIDENCE,NIGHT" ]
arrPREM_DESC_TYPE = ["BAR/NIGHT CLUB", "STREET", "nonsence", "RESIDENCE-HOUSE" ]
arrLAW_CAT_CD = [ "FELONY", "nonsense", "VIOLATION", "MISDEMEANOR" ]

In [7]:
ARRrowDataToWrite = []
for currPedesBoroBeingProcessed in arrPedesDbBoroughNames[:1]:
    for currPedesYearBeingProcessed in arrPedesYear[:1]:
        for currPedesMonthBeingProcessed in arrPedesMonth[:1]:
            print(f"\n\n*** **** **** ****\nProcessing for:\nYear={currPedesYearBeingProcessed}\tMonth={currPedesMonthBeingProcessed}\t\tBoro={currPedesBoroBeingProcessed}")
            #logging.warning(f"\n\n*** **** **** ****\nProcessing for:\nYear={currPedesYearBeingProcessed}\tMonth={currPedesMonthBeingProcessed}\t\tBoro={currPedesBoroBeingProcessed}")
            rowDataToWrite = []
            rowDataToWrite.append(currPedesYearBeingProcessed)
            rowDataToWrite.append(currPedesMonthBeingProcessed)
            rowDataToWrite.append(currPedesBoroBeingProcessed)
            
            # read the count of pedestrians and store it
            # -- db.coll2pedesdata.aggregate([ {"$match": {"year": 2007, "month": 5, "boroughBridgeName": "Queens"} } , {"$group": {"_id": 1, "TotalPedesCount": {"$sum": "$count"}}} ]) --- shows 199427 count of pedes summed
            pipeline1 = [ {"$match": {"year": 0, "month": 0, "boroughBridgeName": "garbage"} } , {"$group": {"_id": 1, "TotalPedesCount": {"$sum": "$count"}}} ]

            pipeline1[0]['$match']['year'] = currPedesYearBeingProcessed
            pipeline1[0]['$match']['month'] = currPedesMonthBeingProcessed
            pipeline1[0]['$match']['boroughBridgeName'] = currPedesBoroBeingProcessed
            cursorTotalPedesCount = coll2PedesData.aggregate(pipeline1)
            for cursor in cursorTotalPedesCount:
                rowDataToWrite.append(cursor["TotalPedesCount"])
            cursorTotalPedesCount.close()
            print(f"Cur 1 done")
            
            # process the ports data now
            # setup and read the port passenger entry count and store it
            # -- db.coll3portsdata.aggregate([ {"$match": {"portName": "new york", "year" : 2007, "month" : 5} } ]) --- get the document for May'2007
            for currPortBeingProcessed in arrPortsDbPortNames:
                pipeline2 = [ {"$match": {"portName": "garbage", "year" : 0, "month" : 0} } ] 
                pipeline2[0]['$match']['year'] = currPedesYearBeingProcessed
                pipeline2[0]['$match']['month'] = currPedesMonthBeingProcessed
                pipeline2[0]['$match']['portName'] = currPortBeingProcessed
                rowDataToWrite.append(currPortBeingProcessed)
                cursorTotalPortEntryCount = coll3PortData.aggregate(pipeline2)
                for cursor in cursorTotalPortEntryCount:
                    rowDataToWrite.append(cursor["count"])
                cursorTotalPortEntryCount.close()
            print(f"Cur 2 done")
            
            # process for the total count of crimes in the month, year, boro and store it
            # setup process total crimes count for the month, year, boro
            # -- db.coll1nyccrimedata.find({"BORO_NM": "QUEENS", "CMPLNT_FR_DT": {$regex: /^05[/][0-9]{2}[/]2007$/}}).count()   -- gave answer 9214
            # -- db.coll1nyccrimedata.aggregate([ {"$match": {"BORO_NM": "QUEENS", "CMPLNT_FR_DT": {$regex: /^05[/][0-9]{2}[/]2007$/} } } , {"$group": {"_id": 1, "TotalCrimesForMonthYearBoro": {"$sum": 1}}} ])  -- gave answer { "_id" : 1, "TotalCrimesForMonthYearBoro" : 9214 }
            pipeline3 = [ {"$match": {"BORO_NM": "garbage", "CMPLNT_FR_DT": {"$regex": "garbage"} } } , {"$group": {"_id": 1, "TotalCrimesForMonthYearBoro": {"$sum": 1}}} ]
            #regexString = '/^' + mymonth + '[/][\d]{2}[/]' + myyear + '$/'
            #newRegexString = '/^' + str(currPedesMonthBeingProcessed).zfill(2) + '[/][0-9]{2}[/]' + str(currPedesYearBeingProcessed) + '$/'
            newRegexString = '^' + str(currPedesMonthBeingProcessed).zfill(2) + '[/][0-9]{2}[/]' + str(currPedesYearBeingProcessed) + '$'
            #print(f"\nnewRegexString=\n{newRegexString}")
            #print(f"before pipeline3=\n{pipeline3}")
            pipeline3[0]['$match']['CMPLNT_FR_DT']['$regex'] = newRegexString
            pipeline3[0]['$match']['BORO_NM'] = currPedesBoroBeingProcessed.upper()
            #print(f"query pipeline3=\n{pipeline3}")
            cursorTotalCrimesInMonthYearBoroCount = coll1CrimeData.aggregate(pipeline3)
            for cursor in cursorTotalCrimesInMonthYearBoroCount:
                rowDataToWrite.append(cursor["TotalCrimesForMonthYearBoro"])
            cursorTotalCrimesInMonthYearBoroCount.close()
            print(f"Cur 3 done")
            
            # setup and process for month, year, boro and KY_CD and store it
            # db.coll1nyccrimedata.aggregate([ {"$match": {"KY_CD": 106, "BORO_NM": "QUEENS", "CMPLNT_FR_DT": {"$regex": "^05[/][0-9]{2}[/]2007$" } } } , {"$group": {"_id": 1, "TotalCrimesForKY_CDForMonthYearBoro": {"$sum": 1}}} ]) --- get the document for May'2007, specific boro, for the specific KY_CD   -- get answer = { "_id" : 1, "TotalCrimesForKY_CDForMonthYearBoro" : 324 }
            # db.coll1nyccrimedata.aggregate( [ {"$match": {"KY_CD": 106, "BORO_NM": "QUEENS", "CMPLNT_FR_DT": {"$regex": "^05[/][0-9]{2}[/]2007$" } } } , {"$group": {"_id": 1, "TotalCrimesForKY_CDForMonthYearBoro": {"$sum": 1}}} ] )
            # db.coll1nyccrimedata.distinct("BORO_NM", {"KY_CD": 106, "CMPLNT_FR_DT": {$regex: '^05[/][0-9]{2}[/]2007$' } })  -- [ "QUEENS", "BROOKLYN", "MANHATTAN", "BRONX", null, "STATEN ISLAND" ]
            for currKY_CDBeingProcessed in arrKY_CD:
                print(f"\nProcessing for:\nKY_CD={currKY_CDBeingProcessed}")
                #logging.warning(f"\nProcessing for:\nKY_CD={currKY_CDBeingProcessed}")
                pipeline4 = [ {"$match": {"KY_CD": 0, "BORO_NM": "garbage", "CMPLNT_FR_DT": {"$regex": "garbage" } } } , {"$group": {"_id": 1, "TotalCrimesForKY_CDForMonthYearBoro": {"$sum": 1}}} ]
                newRegexString = '^' + str(currPedesMonthBeingProcessed).zfill(2) + '[/][0-9]{2}[/]' + str(currPedesYearBeingProcessed) + '$'
                pipeline4[0]['$match']['CMPLNT_FR_DT']['$regex'] = newRegexString
                pipeline4[0]['$match']['BORO_NM'] = currPedesBoroBeingProcessed.upper()
                pipeline4[0]['$match']['KY_CD'] = currKY_CDBeingProcessed
                print(f"query pipeline4=\n{pipeline4}")
                rowDataToWrite.append(currKY_CDBeingProcessed)
                cursorTotalCrimesInKY_CDMonthYearBoroCount = coll1CrimeData.aggregate(pipeline4)
                toAppendValue = 0
                for cursor in cursorTotalCrimesInKY_CDMonthYearBoroCount:
                    print(f"     entered for cursor\t cursor={cursor}")
                    try:
                        toAppendValue = cursor["TotalCrimesForKY_CDForMonthYearBoro"]
                    except KeyError:
                        print(f"\n\t\tKeyError for KY_CD={currKY_CDBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        print(f"\t\tSo inserted 0 for the crime count and continued.")
                        #logging.warning(f"\n\t\tKeyError for KY_CD={currKY_CDBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        #logging.warning(f"\n\t\tSo inserted 0 for the crime count and continued.")
                        toAppendValue = 0
                    except Exception as e:
                        cursorTotalCrimesInKY_CDMonthYearBoroCount.close()
                        print(f"\nERROR: Unhandled exception processing for:\n\t\tKeyError for KY_CD={currKY_CDBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        print(f"ERROR: Error details = {e}")
                        #logging.error(f"\nERROR: Unhandled exception processing for:\n\t\tKeyError for KY_CD={currKY_CDBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        #logging.error(f"ERROR: Error details = {e}")
                    cursorTotalCrimesInKY_CDMonthYearBoroCount.close()
                rowDataToWrite.append(toAppendValue)
            print(f"Cur 4 done")
            
            # process the OFNS_DESC data and store it -- pipeline 5
            # setup and process for month, year, boro and OFNS_DESC and store it
            # db.coll1nyccrimedata.aggregate([ {"$match": {"OFNS_DESC": "FELONY ASSAULT", "BORO_NM": "QUEENS", "CMPLNT_FR_DT": {"$regex": "^05[/][0-9]{2}[/]2007$" } } } , {"$group": {"_id": 1, "TotalCrimesForOFNS_DESCForMonthYearBoro": {"$sum": 1}}} ]) --- get the document for May'2007, specific boro, for the specific OFNS_DESC   -- get answer = { "_id" : 1, "TotalCrimesForOFNS_DESCForMonthYearBoro" : 324 }
            # db.coll1nyccrimedata.distinct("OFNS_DESC", {"BORO_NM": "QUEENS", "CMPLNT_FR_DT": {$regex: '^05[/][0-9]{2}[/]2007$' } })  -- answer = arrOFNS_DESC = ['FELONY ASSAULT', 'HARRASSMENT 2', 'BURGLARY', and more items totalling to 40 approx ]
            for currOFNS_DESCBeingProcessed in arrOFNS_DESC:
                print(f"\nProcessing for:\nOFNS_DESC={currOFNS_DESCBeingProcessed}")
                #logging.warning(f"\nProcessing for:\nOFNS_DESC={currOFNS_DESCBeingProcessed}")
                pipeline5 = [ {"$match": {"OFNS_DESC": "garbage", "BORO_NM": "garbage", "CMPLNT_FR_DT": {"$regex": "garbage" } } } , {"$group": {"_id": 1, "TotalCrimesForOFNS_DESCForMonthYearBoro": {"$sum": 1}}} ]
                newRegexString = '^' + str(currPedesMonthBeingProcessed).zfill(2) + '[/][0-9]{2}[/]' + str(currPedesYearBeingProcessed) + '$'
                pipeline5[0]['$match']['CMPLNT_FR_DT']['$regex'] = newRegexString
                pipeline5[0]['$match']['BORO_NM'] = currPedesBoroBeingProcessed.upper()
                pipeline5[0]['$match']['OFNS_DESC'] = currOFNS_DESCBeingProcessed
                print(f"query pipeline5=\n{pipeline5}")
                rowDataToWrite.append(currOFNS_DESCBeingProcessed)
                cursorTotalCrimesInOFNS_DESCMonthYearBoroCount = coll1CrimeData.aggregate(pipeline5)
                toAppendValue = 0
                for cursor in cursorTotalCrimesInOFNS_DESCMonthYearBoroCount:
                    print(f"     entered for cursor\t cursor={cursor}")
                    try:
                        toAppendValue = cursor["TotalCrimesForOFNS_DESCForMonthYearBoro"]
                    except KeyError:
                        print(f"\n\t\tKeyError for OFNS_DESC={currOFNS_DESCBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        print(f"\t\tSo inserted 0 for the crime count and continued.")
                        #logging.warning(f"\n\t\tKeyError for OFNS_DESC={currOFNS_DESCBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        #logging.warning(f"\n\t\tSo inserted 0 for the crime count and continued.")
                        toAppendValue = 0
                    except Exception as e:
                        cursorTotalCrimesInOFNS_DESCMonthYearBoroCount.close()
                        print(f"\nERROR: Unhandled exception processing for:\n\t\tKeyError for OFNS_DESC={currOFNS_DESCBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        print(f"ERROR: Error details = {e}")
                        logging.error(f"\nERROR: Unhandled exception processing for:\n\t\tKeyError for OFNS_DESC={currOFNS_DESCBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        logging.error(f"ERROR: Error details = {e}")
                cursorTotalCrimesInOFNS_DESCMonthYearBoroCount.close()
                rowDataToWrite.append(toAppendValue)
            print(f"Cur 5 done")
            
            # process the PD_CD data and store it -- pipeline 6
            # setup and process for month, year, boro and PD_CD and store it
            # db.coll1nyccrimedata.aggregate([ {"$match": {"PD_CD": 109, "BORO_NM": "QUEENS", "CMPLNT_FR_DT": {"$regex": "^05[/][0-9]{2}[/]2007$" } } } , {"$group": {"_id": 1, "TotalCrimesForPD_CDForMonthYearBoro": {"$sum": 1}}} ]) --- get the document for May'2007, specific boro, for the specific PD_CD   -- get answer = { "_id" : 1, "TotalCrimesForPD_CDForMonthYearBoro" : 282 }
            # db.coll1nyccrimedata.distinct("PD_CD", {"BORO_NM": "QUEENS", "CMPLNT_FR_DT": {$regex: '^05[/][0-9]{2}[/]2007$' } })  -- answer = [ 109, 637, 638, 223, and many more items ]
            for currPD_CDBeingProcessed in arrPD_CD:
                print(f"\nProcessing for:\nPD_CD={currPD_CDBeingProcessed}")
                #logging.warning(f"\nProcessing for:\nPD_CD={currPD_CDBeingProcessed}")
                pipeline6 = [ {"$match": {"PD_CD": 0, "BORO_NM": "garbage", "CMPLNT_FR_DT": {"$regex": "garbage" } } } , {"$group": {"_id": 1, "TotalCrimesForPD_CDForMonthYearBoro": {"$sum": 1}}} ]
                newRegexString = '^' + str(currPedesMonthBeingProcessed).zfill(2) + '[/][0-9]{2}[/]' + str(currPedesYearBeingProcessed) + '$'
                pipeline6[0]['$match']['CMPLNT_FR_DT']['$regex'] = newRegexString
                pipeline6[0]['$match']['BORO_NM'] = currPedesBoroBeingProcessed.upper()
                pipeline6[0]['$match']['PD_CD'] = currPD_CDBeingProcessed
                print(f"query pipeline6=\n{pipeline6}")
                rowDataToWrite.append(currPD_CDBeingProcessed)
                cursorTotalCrimesInPD_CDMonthYearBoroCount = coll1CrimeData.aggregate(pipeline6)
                toAppendValue = 0
                for cursor in cursorTotalCrimesInPD_CDMonthYearBoroCount:
                    print(f"     entered for cursor\t cursor={cursor}")
                    try:
                        toAppendValue = cursor["TotalCrimesForPD_CDForMonthYearBoro"]
                    except KeyError:
                        print(f"\n\t\tKeyError for PD_CD={currPD_CDBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        print(f"\t\tSo inserted 0 for the crime count and continued.")
                        #logging.warning(f"\n\t\tKeyError for PD_CD={currPD_CDBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        #logging.warning(f"\n\t\tSo inserted 0 for the crime count and continued.")
                        toAppendValue = 0
                    except Exception as e:
                        cursorTotalCrimesInPD_CDMonthYearBoroCount.close()
                        print(f"\nERROR: Unhandled exception processing for:\n\t\tKeyError for PD_CD={currPD_CDBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        print(f"ERROR: Error details = {e}")
                        #logging.error(f"\nERROR: Unhandled exception processing for:\n\t\tKeyError for PD_CD={currPD_CDBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        #logging.error(f"ERROR: Error details = {e}")
                cursorTotalCrimesInPD_CDMonthYearBoroCount.close()
                rowDataToWrite.append(toAppendValue)
            print(f"Cur 6 done")
            
            # process the PD_DESC data and store it -- pipeline 7
            # setup and process for month, year, boro and PD_DESC and store it
            # db.coll1nyccrimedata.aggregate([ {"$match": {"PD_DESC": "ASSAULT 2,1,UNCLASSIFIED", "BORO_NM": "QUEENS", "CMPLNT_FR_DT": {"$regex": "^05[/][0-9]{2}[/]2007$" } } } , {"$group": {"_id": 1, "TotalCrimesForPD_DESCForMonthYearBoro": {"$sum": 1}}} ]) --- get the document for May'2007, specific boro, for the specific PD_DESC   -- get answer = { "_id" : 1, "TotalCrimesForPD_DESCForMonthYearBoro" : 282 }
            # db.coll1nyccrimedata.distinct("PD_DESC", {"BORO_NM": "QUEENS", "CMPLNT_FR_DT": {$regex: '^05[/][0-9]{2}[/]2007$' } })  -- answer = [ "ASSAULT 2,1,UNCLASSIFIED", "HARASSMENT,SUBD 1,CIVILIAN", "BURGLARY,RESIDENCE,NIGHT", and many more items ]
            for currPD_DESCBeingProcessed in arrPD_DESC:
                print(f"\nProcessing for:\nPD_DESC={currPD_DESCBeingProcessed}")
                #logging.warning(f"\nProcessing for:\nPD_DESC={currPD_DESCBeingProcessed}")
                pipeline7 = [ {"$match": {"PD_DESC": "garbage", "BORO_NM": "garbage", "CMPLNT_FR_DT": {"$regex": "garbage" } } } , {"$group": {"_id": 1, "TotalCrimesForPD_DESCForMonthYearBoro": {"$sum": 1}}} ]
                newRegexString = '^' + str(currPedesMonthBeingProcessed).zfill(2) + '[/][0-9]{2}[/]' + str(currPedesYearBeingProcessed) + '$'
                pipeline7[0]['$match']['CMPLNT_FR_DT']['$regex'] = newRegexString
                pipeline7[0]['$match']['BORO_NM'] = currPedesBoroBeingProcessed.upper()
                pipeline7[0]['$match']['PD_DESC'] = currPD_DESCBeingProcessed
                #print(f"query pipeline7=\n{pipeline7}")
                rowDataToWrite.append(currPD_DESCBeingProcessed)
                cursorTotalCrimesInPD_DESCMonthYearBoroCount = coll1CrimeData.aggregate(pipeline7)
                toAppendValue = 0
                for cursor in cursorTotalCrimesInPD_DESCMonthYearBoroCount:
                    print(f"     entered for cursor\t cursor={cursor}")
                    try:
                        toAppendValue = cursor["TotalCrimesForPD_DESCForMonthYearBoro"]
                    except KeyError:
                        print(f"\n\t\tKeyError for PD_DESC={currPD_DESCBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        print(f"\t\tSo inserted 0 for the crime count and continued.")
                        #logging.warning(f"\n\t\tKeyError for PD_DESC={currPD_DESCBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        #logging.warning(f"\n\t\tSo inserted 0 for the crime count and continued.")
                        toAppendValue = 0
                    except Exception as e:
                        cursorTotalCrimesInPD_DESCMonthYearBoroCount.close()
                        print(f"\nERROR: Unhandled exception processing for:\n\t\tKeyError for PD_DESC={currPD_DESCBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        print(f"ERROR: Error details = {e}")
                        #logging.error(f"\nERROR: Unhandled exception processing for:\n\t\tKeyError for PD_DESC={currPD_DESCBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        #logging.error(f"ERROR: Error details = {e}")
                cursorTotalCrimesInPD_DESCMonthYearBoroCount.close()
                rowDataToWrite.append(toAppendValue)
            print(f"Cur 7 done")
            #
            # process the PREM_TYP_DESC data and store it -- pipeline 8
            # setup and process for month, year, boro and PREM_TYP_DESC and store it
            # db.coll1nyccrimedata.aggregate([ {"$match": {"PREM_TYP_DESC": "STREET", "BORO_NM": "QUEENS", "CMPLNT_FR_DT": {"$regex": "^05[/][0-9]{2}[/]2007$" } } } , {"$group": {"_id": 1, "TotalCrimesForPREM_TYP_DESCForMonthYearBoro": {"$sum": 1}}} ]) --- get the document for May'2007, specific boro, for the specific PREM_TYP_DESC   -- get answer = { "_id" : 1, "TotalCrimesForPREM_TYP_DESCForMonthYearBoro" : 3159 }
            # db.coll1nyccrimedata.distinct("PREM_TYP_DESC", {"BORO_NM": "QUEENS", "CMPLNT_FR_DT": {$regex: '^05[/][0-9]{2}[/]2007$' } })  -- answer = ["BAR/NIGHT CLUB", "STREET", "RESIDENCE-HOUSE", and many more items ]
            for currPREM_TYP_DESCBeingProcessed in arrPREM_DESC_TYPE:
                print(f"\nProcessing for:\nPREM_TYP_DESC={currPREM_TYP_DESCBeingProcessed}")
                #logging.warning(f"\nProcessing for:\nPREM_TYP_DESC={currPREM_TYP_DESCBeingProcessed}")
                pipeline8 = [ {"$match": {"PREM_TYP_DESC": "garbage", "BORO_NM": "garbage", "CMPLNT_FR_DT": {"$regex": "garbage" } } } , {"$group": {"_id": 1, "TotalCrimesForPREM_TYP_DESCForMonthYearBoro": {"$sum": 1}}} ]
                newRegexString = '^' + str(currPedesMonthBeingProcessed).zfill(2) + '[/][0-9]{2}[/]' + str(currPedesYearBeingProcessed) + '$'
                pipeline8[0]['$match']['CMPLNT_FR_DT']['$regex'] = newRegexString
                pipeline8[0]['$match']['BORO_NM'] = currPedesBoroBeingProcessed.upper()
                pipeline8[0]['$match']['PREM_TYP_DESC'] = currPREM_TYP_DESCBeingProcessed
                #print(f"query pipeline8=\n{pipeline8}")
                rowDataToWrite.append(currPREM_TYP_DESCBeingProcessed)
                cursorTotalCrimesInPREM_TYP_DESCMonthYearBoroCount = coll1CrimeData.aggregate(pipeline8)
                toAppendValue = 0
                for cursor in cursorTotalCrimesInPREM_TYP_DESCMonthYearBoroCount:
                    print(f"     entered for cursor\t cursor={cursor}")
                    try:
                        toAppendValue = cursor["TotalCrimesForPREM_TYP_DESCForMonthYearBoro"]
                    except KeyError:
                        print(f"\n\t\tKeyError for PREM_TYP_DESC={currPREM_TYP_DESCBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        print(f"\t\tSo inserted 0 for the crime count and continued.")
                        #logging.warning(f"\n\t\tKeyError for PREM_TYP_DESC={currPREM_TYP_DESCBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        #logging.warning(f"\n\t\tSo inserted 0 for the crime count and continued.")
                        toAppendValue = 0
                    except Exception as e:
                        cursorTotalCrimesInPREM_TYP_DESCMonthYearBoroCount.close()
                        print(f"\nERROR: Unhandled exception processing for:\n\t\tKeyError for PREM_TYP_DESC={currPREM_TYP_DESCBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        print(f"ERROR: Error details = {e}")
                        #logging.error(f"\nERROR: Unhandled exception processing for:\n\t\tKeyError for PREM_TYP_DESC={currPREM_TYP_DESCBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        #logging.error(f"ERROR: Error details = {e}")
                cursorTotalCrimesInPREM_TYP_DESCMonthYearBoroCount.close()
                rowDataToWrite.append(toAppendValue)
            print(f"Cur 8 done")
            
            # process the LAW_CAT_CD data and store it -- pipeline 9
            # setup and process for month, year, boro and LAW_CAT_CD and store it
            # db.coll1nyccrimedata.aggregate([ {"$match": {"LAW_CAT_CD": "FELONY", "BORO_NM": "QUEENS", "CMPLNT_FR_DT": {"$regex": "^05[/][0-9]{2}[/]2007$" } } } , {"$group": {"_id": 1, "TotalCrimesForLAW_CAT_CDForMonthYearBoro": {"$sum": 1}}} ]) --- get the document for May'2007, specific boro, for the specific LAW_CAT_CD   -- get answer = { "_id" : 1, "TotalCrimesForLAW_CAT_CDForMonthYearBoro" : 3006 }
            # db.coll1nyccrimedata.distinct("LAW_CAT_CD", {"BORO_NM": "QUEENS", "CMPLNT_FR_DT": {$regex: '^05[/][0-9]{2}[/]2007$' } })  -- answer = [ "FELONY", "VIOLATION", "MISDEMEANOR" ]
            for currLAW_CAT_CDBeingProcessed in arrLAW_CAT_CD:
                print(f"\nProcessing for:\nLAW_CAT_CD={currLAW_CAT_CDBeingProcessed}")
                #logging.warning(f"\nProcessing for:\nLAW_CAT_CD={currLAW_CAT_CDBeingProcessed}")
                pipeline9 = [ {"$match": {"LAW_CAT_CD": "garbage", "BORO_NM": "garbage", "CMPLNT_FR_DT": {"$regex": "garbage" } } } , {"$group": {"_id": 1, "TotalCrimesForLAW_CAT_CDForMonthYearBoro": {"$sum": 1}}} ]
                newRegexString = '^' + str(currPedesMonthBeingProcessed).zfill(2) + '[/][0-9]{2}[/]' + str(currPedesYearBeingProcessed) + '$'
                pipeline9[0]['$match']['CMPLNT_FR_DT']['$regex'] = newRegexString
                pipeline9[0]['$match']['BORO_NM'] = currPedesBoroBeingProcessed.upper()
                pipeline9[0]['$match']['LAW_CAT_CD'] = currLAW_CAT_CDBeingProcessed
                #print(f"query pipeline9=\n{pipeline9}")
                rowDataToWrite.append(currLAW_CAT_CDBeingProcessed)
                cursorTotalCrimesInLAW_CAT_CDMonthYearBoroCount = coll1CrimeData.aggregate(pipeline9)
                toAppendValue = 0
                for cursor in cursorTotalCrimesInLAW_CAT_CDMonthYearBoroCount:
                    print(f"     entered for cursor\t cursor={cursor}")
                    try:
                        toAppendValue = cursor["TotalCrimesForLAW_CAT_CDForMonthYearBoro"]
                    except KeyError:
                        print(f"\n\t\tKeyError for LAW_CAT_CD={currLAW_CAT_CDBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        print(f"\t\tSo inserted 0 for the crime count and continued.")
                        #logging.warning(f"\n\t\tKeyError for LAW_CAT_CD={currLAW_CAT_CDBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        #logging.warning(f"\n\t\tSo inserted 0 for the crime count and continued.")
                        toAppendValue = 0
                    except Exception as e:
                        cursorTotalCrimesInLAW_CAT_CDMonthYearBoroCount.close()
                        print(f"\nERROR: Unhandled exception processing for:\n\t\tKeyError for LAW_CAT_CD={currLAW_CAT_CDBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        print(f"ERROR: Error details = {e}")
                        #logging.error(f"\nERROR: Unhandled exception processing for:\n\t\tKeyError for LAW_CAT_CD={currLAW_CAT_CDBeingProcessed} , Year={currPedesYearBeingProcessed} , Month={currPedesMonthBeingProcessed} , Boro={currPedesBoroBeingProcessed.upper()}")
                        #logging.error(f"ERROR: Error details = {e}")
                cursorTotalCrimesInLAW_CAT_CDMonthYearBoroCount.close()
                rowDataToWrite.append(toAppendValue)
            print(f"Cur 9 done")
            #####################
            ARRrowDataToWrite.append(rowDataToWrite)
   



*** **** **** ****
Processing for:
Year=2007	Month=5		Boro=Queens
Cur 1 done
Cur 2 done
Cur 3 done

Processing for:
KY_CD=106
query pipeline4=
[{'$match': {'KY_CD': 106, 'BORO_NM': 'QUEENS', 'CMPLNT_FR_DT': {'$regex': '^05[/][0-9]{2}[/]2007$'}}}, {'$group': {'_id': 1, 'TotalCrimesForKY_CDForMonthYearBoro': {'$sum': 1}}}]
     entered for cursor	 cursor={'_id': 1, 'TotalCrimesForKY_CDForMonthYearBoro': 324}

Processing for:
KY_CD=101
query pipeline4=
[{'$match': {'KY_CD': 101, 'BORO_NM': 'QUEENS', 'CMPLNT_FR_DT': {'$regex': '^05[/][0-9]{2}[/]2007$'}}}, {'$group': {'_id': 1, 'TotalCrimesForKY_CDForMonthYearBoro': {'$sum': 1}}}]
Cur 4 done

Processing for:
OFNS_DESC=FELONY ASSAULT
query pipeline5=
[{'$match': {'OFNS_DESC': 'FELONY ASSAULT', 'BORO_NM': 'QUEENS', 'CMPLNT_FR_DT': {'$regex': '^05[/][0-9]{2}[/]2007$'}}}, {'$group': {'_id': 1, 'TotalCrimesForOFNS_DESCForMonthYearBoro': {'$sum': 1}}}]
     entered for cursor	 cursor={'_id': 1, 'TotalCrimesForOFNS_DESCForMonthYearBoro': 324}
Cu

In [8]:
print(ARRrowDataToWrite)

[[2007, 5, 'Queens', 199247, 'new york', 306510, 'newark', 118361, 'los angeles', 196387, 9214, 106, 324, 101, 0, 'FELONY ASSAULT', 324, 109, 282, 637, 532, 'ASSAULT 2,1,UNCLASSIFIED', 282, 'BAR/NIGHT CLUB', 67, 'STREET', 3159, 'nonsence', 0, 'RESIDENCE-HOUSE', 2051, 'FELONY', 3006, 'nonsense', 0, 'VIOLATION', 1193, 'MISDEMEANOR', 5015]]
